In [56]:
import os 
import glob
import pvlib
from bs4 import BeautifulSoup as bs
import re
import requests
import shutil
import pandas as pd
import proplot as ppl

In [26]:
tmy2_files = glob.glob(os.path.join("/Users/jmccarty/Nextcloud/Projects/13_CISBAT23_idf/tmy2","*.epw"))

In [38]:
wmo_code_dict = {}

for fname in tmy2_files:
    with open(fname, "r") as fp:
        cz = fname.split(os.sep)[-1].split("_")[0]
        tmy2 = fp.readline()
        wmo_code = tmy2.split(",")[5]
        wmo_code_dict.update({wmo_code:cz})

In [39]:
tmy_3_files = []

for dir in glob.glob(os.path.join("/Users/jmccarty/Nextcloud/Projects/13_CISBAT23_idf/tmy3_init","*")):
    tmy_3_files.append(glob.glob(os.path.join(dir,"*.epw")))
    
tmy_3_files = [item for sublist in tmy_3_files for item in sublist]


In [55]:
for fname in tmy_3_files:
    fname_elements = fname.split(os.sep)[-1].split(".")
    city = ("_").join(fname_elements[0:2])
    print(city)
    wmo_code = fname_elements[-2].split("_")[0]
    cz = wmo_code_dict[str(wmo_code)]
    dst = os.path.join("/Users/jmccarty/Nextcloud/Projects/13_CISBAT23_idf/tmy3",f"{cz.lower()}_{city}.epw")
    shutil.copy(fname,dst)

USA_CA_San_Francisco
USA_MD_Baltimore-Washington_Intl
USA_CA_Los_Angeles
USA_NV_Las_Vegas-McCarran
USA_MN_Minneapolis-St_Paul
USA_TX_Houston-Bush_Intercontinental
USA_IL_Chicago_OHare
USA_AZ_Phoenix-Sky_Harbor
USA_MT_Helena_Rgnl
USA_NM_Albuquerque_Intl
USA_FL_Miami_Intl
USA_CO_Boulder-Table_Mountain
USA_GA_Atlanta-Hartsfield-Jackson_Intl
USA_MN_Duluth_Intl
USA_AK_Fairbanks_Intl
USA_WA_Seattle-Tacoma_Intl
